Here goes the code to generate the integrated object with all cell types and to compute the DR space and the FeaturePlot gene expression plots

In [1]:
library(tidyverse)
library(Seurat)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.1 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
data_dir <- "~/PhD/S_210125_CRCTME_Jan21/CRCTME_story/Integration_CRCTME-Aug21/RData/"

# Data ingestion and preparation

* Prepare data: 
  * List of Seurat objects.
    * Ensure all have SCT.
  * Apply same QC to all (not as strict as the one in INTepi21)